<a href="https://colab.research.google.com/github/abhishekmishra-bareilly/Deep-Learning/blob/main/Keras_Hyperparameter_Tuning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [43]:
import numpy as np
import pandas as pd


## Data overview

In [44]:
data = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/Deep Learning/learn outside/diabetes.csv')

In [45]:
data.head()

,Pregnancies,Glucose,BloodPressure,SkinThickness,Insulin,BMI,DiabetesPedigreeFunction,Age,Outcome
0,6,148,72,35,0,33.6,0.627,50,1
1,1,85,66,29,0,26.6,0.351,31,0
2,8,183,64,0,0,23.3,0.672,32,1
3,1,89,66,23,94,28.1,0.167,21,0
4,0,137,40,35,168,43.1,2.288,33,1


In [46]:
# Check the corelation
data.corr()['Outcome']

Pregnancies                 0.221898
Glucose                     0.466581
BloodPressure               0.065068
SkinThickness               0.074752
Insulin                     0.130548
BMI                         0.292695
DiabetesPedigreeFunction    0.173844
Age                         0.238356
Outcome                     1.000000
Name: Outcome, dtype: float64

In [47]:
x = data.iloc[:,:-1]
y = data.iloc[:,-1]

In [48]:
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
x = scaler.fit_transform(x)
x

array([[ 0.63994726,  0.84832379,  0.14964075, ...,  0.20401277,
         0.46849198,  1.4259954 ],
       [-0.84488505, -1.12339636, -0.16054575, ..., -0.68442195,
        -0.36506078, -0.19067191],
       [ 1.23388019,  1.94372388, -0.26394125, ..., -1.10325546,
         0.60439732, -0.10558415],
       ...,
       [ 0.3429808 ,  0.00330087,  0.14964075, ..., -0.73518964,
        -0.68519336, -0.27575966],
       [-0.84488505,  0.1597866 , -0.47073225, ..., -0.24020459,
        -0.37110101,  1.17073215],
       [-0.84488505, -0.8730192 ,  0.04624525, ..., -0.20212881,
        -0.47378505, -0.87137393]])

In [49]:
from sklearn.model_selection import train_test_split

x_train,x_test,y_train,y_test = train_test_split(x,y,test_size=0.2,random_state=1)

## ANN model implimentation

In [115]:
import tensorflow
from tensorflow import keras
from keras import Sequential
from keras.layers import Dense,Dropout


In [51]:
# Model implimentation
model = Sequential()

#adding model dense layers

model.add(Dense(32,activation='relu',input_dim=8))

model.add(Dense(1,activation='sigmoid'))


# Model summary
model.summary()

# modelcompilation
model.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy'])

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_4 (Dense)             (None, 32)                288       
                                                                 
 dense_5 (Dense)             (None, 1)                 33        
                                                                 
Total params: 321
Trainable params: 321
Non-trainable params: 0
_________________________________________________________________


In [52]:
history = model.fit(x_train,y_train,batch_size=32,epochs=10,validation_data=(x_test,y_test))

Epoch 1/10
20/20 [==============================] - 1s 10ms/step - loss: 0.9128 - accuracy: 0.3518 - val_loss: 0.8136 - val_accuracy: 0.3571
Epoch 2/10
20/20 [==============================] - 0s 4ms/step - loss: 0.7905 - accuracy: 0.3518 - val_loss: 0.7196 - val_accuracy: 0.4416
Epoch 3/10
20/20 [==============================] - 0s 3ms/step - loss: 0.7073 - accuracy: 0.4723 - val_loss: 0.6509 - val_accuracy: 0.6558
Epoch 4/10
20/20 [==============================] - 0s 4ms/step - loss: 0.6472 - accuracy: 0.6482 - val_loss: 0.6038 - val_accuracy: 0.7532
Epoch 5/10
20/20 [==============================] - 0s 3ms/step - loss: 0.6044 - accuracy: 0.7231 - val_loss: 0.5687 - val_accuracy: 0.7792
Epoch 6/10
20/20 [==============================] - 0s 3ms/step - loss: 0.5731 - accuracy: 0.7476 - val_loss: 0.5437 - val_accuracy: 0.7987
Epoch 7/10
20/20 [==============================] - 0s 4ms/step - loss: 0.5510 - accuracy: 0.7476 - val_loss: 0.5237 - val_accuracy: 0.8117
Epoch 8/10
20/20 [=

## 1. how to select best optimizer
## 2. no.of nodes in a layer
## 3. how to select no of layers
## 4. all in one model

## **1. How to select best optimizer**

In [53]:
!pip install -q -U keras-tuner

In [54]:
import keras_tuner as kt

In [55]:
def build_model(hp):
  # Model implimentation
  model = Sequential()

  #adding model dense layers

  model.add(Dense(32,activation='relu',input_dim=8))

  model.add(Dense(1,activation='sigmoid'))

  # set values of optimizers
  optimizer = hp.Choice('optimizer', values = ['adam','sgd','rmsprop','adadelta'])


  model.compile(optimizer=optimizer, loss='binary_crossentropy',metrics=['accuracy'])

  return model

In [56]:
# Creating tuner
tuner = kt.RandomSearch(build_model, objective = 'val_accuracy', max_trials = 5)

In [57]:
tuner.search(x_train, y_train, epochs=5, validation_data=(x_test,y_test))

In [60]:
# Getting best optimizer values
tuner.get_best_hyperparameters()[0].values

{'optimizer': 'rmsprop'}

In [62]:
model = tuner.get_best_models(num_models=1)[0]

In [63]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 32)                288       
                                                                 
 dense_1 (Dense)             (None, 1)                 33        
                                                                 
Total params: 321
Trainable params: 321
Non-trainable params: 0
_________________________________________________________________


In [65]:
model.fit(x_train, y_train, batch_size=32, epochs=100, initial_epoch = 6, validation_data=(x_test,y_test))

Epoch 7/100
20/20 [==============================] - 1s 10ms/step - loss: 0.5202 - accuracy: 0.7362 - val_loss: 0.5045 - val_accuracy: 0.7922
Epoch 8/100
20/20 [==============================] - 0s 4ms/step - loss: 0.5047 - accuracy: 0.7427 - val_loss: 0.4924 - val_accuracy: 0.7987
Epoch 9/100
20/20 [==============================] - 0s 4ms/step - loss: 0.4931 - accuracy: 0.7508 - val_loss: 0.4839 - val_accuracy: 0.7922
Epoch 10/100
20/20 [==============================] - 0s 3ms/step - loss: 0.4843 - accuracy: 0.7671 - val_loss: 0.4775 - val_accuracy: 0.7987
Epoch 11/100
20/20 [==============================] - 0s 3ms/step - loss: 0.4769 - accuracy: 0.7638 - val_loss: 0.4738 - val_accuracy: 0.7987
Epoch 12/100
20/20 [==============================] - 0s 3ms/step - loss: 0.4709 - accuracy: 0.7752 - val_loss: 0.4718 - val_accuracy: 0.7987
Epoch 13/100
20/20 [==============================] - 0s 4ms/step - loss: 0.4664 - accuracy: 0.7736 - val_loss: 0.4690 - val_accuracy: 0.7987
Epoch 14

## **2. no. of correct nuarans in a layer**

In [85]:
def build_model(hp):
  # Model implimentation
  model = Sequential()

  # set values of optimizers
  units = hp.Int('units', min_value = 8,max_value = 128)

  #adding model dense layers

  model.add(Dense(units = units, activation='relu', input_dim=8))

  model.add(Dense(1,activation='sigmoid'))


  model.compile(optimizer='rmsprop', loss='binary_crossentropy',metrics=['accuracy'])

  return model

In [87]:
# Tuner object
tuner = kt.RandomSearch(build_model, objective = 'val_accuracy', max_trials = 5, directory = 'kk', )

In [88]:
tuner.search(x_train, y_train, epochs=5, validation_data=(x_test,y_test))

Trial 5 Complete [00h 00m 01s]
val_accuracy: 0.7532467246055603

Best val_accuracy So Far: 0.8051947951316833
Total elapsed time: 00h 00m 07s


In [89]:
# Getting best optimizer values
tuner.get_best_hyperparameters()[0].values

{'units': 108}

In [90]:
model = tuner.get_best_models(num_models=1)[0]

In [91]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 108)               972       
                                                                 
 dense_1 (Dense)             (None, 1)                 109       
                                                                 
Total params: 1,081
Trainable params: 1,081
Non-trainable params: 0
_________________________________________________________________


In [ ]:
model.fit(x_train, y_train, batch_size=32, epochs=100, initial_epoch = 6, validation_data=(x_test,y_test))

## **3. how to select no of layers**

In [96]:
def build_model(hp):
  # Model implimentation
  model = Sequential()

  #adding model dense layers
  model.add(Dense(108, activation='relu', input_dim=8))

  for i in range(hp.Int('num_layers', min_value=1, max_value =10)):

    model.add(Dense(108,activation='relu'))

  model.add(Dense(1,activation='sigmoid'))

  # Model compiletion
  model.compile(optimizer='rmsprop', loss='binary_crossentropy',metrics=['accuracy'])


  return model

In [97]:
# Tuner object
tuner = kt.RandomSearch(build_model, objective = 'val_accuracy',
                        max_trials = 3,
                        directory = 'mydr', project_name = 'num_layers')

In [98]:
tuner.search(x_train, y_train, epochs=5, validation_data=(x_test,y_test))

Trial 3 Complete [00h 00m 02s]
val_accuracy: 0.8311688303947449

Best val_accuracy So Far: 0.8311688303947449
Total elapsed time: 00h 00m 07s


In [99]:
# Getting best optimizer values
tuner.get_best_hyperparameters()[0].values

{'num_layers': 2}

In [100]:
model = tuner.get_best_models(num_models=1)[0]

In [101]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 108)               972       
                                                                 
 dense_1 (Dense)             (None, 108)               11772     
                                                                 
 dense_2 (Dense)             (None, 108)               11772     
                                                                 
 dense_3 (Dense)             (None, 1)                 109       
                                                                 
Total params: 24,625
Trainable params: 24,625
Non-trainable params: 0
_________________________________________________________________


In [102]:
model.fit(x_train, y_train, batch_size=32, epochs=100, initial_epoch = 6, validation_data=(x_test,y_test))

Epoch 7/100
20/20 [==============================] - 2s 26ms/step - loss: 0.4362 - accuracy: 0.7915 - val_loss: 0.5046 - val_accuracy: 0.7597
Epoch 8/100
20/20 [==============================] - 0s 14ms/step - loss: 0.4354 - accuracy: 0.7883 - val_loss: 0.4614 - val_accuracy: 0.8182
Epoch 9/100
20/20 [==============================] - 0s 9ms/step - loss: 0.4253 - accuracy: 0.7948 - val_loss: 0.4547 - val_accuracy: 0.7987
Epoch 10/100
20/20 [==============================] - 0s 10ms/step - loss: 0.4228 - accuracy: 0.7915 - val_loss: 0.4610 - val_accuracy: 0.8247
Epoch 11/100
20/20 [==============================] - 0s 8ms/step - loss: 0.4146 - accuracy: 0.8046 - val_loss: 0.4738 - val_accuracy: 0.7922
Epoch 12/100
20/20 [==============================] - 0s 7ms/step - loss: 0.4098 - accuracy: 0.7899 - val_loss: 0.4656 - val_accuracy: 0.8182
Epoch 13/100
20/20 [==============================] - 0s 9ms/step - loss: 0.4035 - accuracy: 0.8029 - val_loss: 0.4577 - val_accuracy: 0.8117
Epoch 

## 4. all in one model

In [129]:
from keras_tuner.protos.keras_tuner_pb2 import Choice
def build_model(hp):
  # Model implimentation
  model = Sequential()

  counter = 0

  #adding model dense layers

  for i in range(hp.Int('num_layers', min_value=1, max_value =10)):
    if counter ==0:

      model.add(Dense(hp.Int('units'+ str(i),min_value=8,max_value=128,step=8),
              activation=hp.Choice('activation' + str(i),values=['relu','tanh','sigmoid']),
              input_dim=8))
      model.add(Dropout(hp.Choice('dropout' + str(i), values=[0.1,0.2,0.3,0.4,0.5,0.6,0.7,0.8,0.9])))
              
              
    else:

      model.add(
          Dense(
              hp.Int('units'+ str(i), min_value = 8,max_value = 128, step = 8),
              activation=hp.Choice('activation' + str(i), values = ['relu','tanh','sigmoid']),
                                          )
              )
      model.add(Dropout(hp.Choice('dropout' + str(i), values =[0.1,0.2,0.3,0.4,0.5,0.6,0.7,0.8,0.9])))

    counter+=1


  model.add(Dense(1,activation='sigmoid'))

  # Model compiletion
  model.compile(optimizer=hp.Choice('optimizer', values = ['rmsprop','adam','sgd','nadam','adadelta']),
                loss='binary_crossentropy',
                metrics=['accuracy'])


  return model

In [130]:
# Tuner object
tuner = kt.RandomSearch(build_model, objective = 'val_accuracy',
                        max_trials = 3,
                        directory = 'mydr', project_name = 'final1')

In [131]:
tuner.search(x_train, y_train, epochs=5, validation_data=(x_test,y_test))

Trial 3 Complete [00h 00m 02s]
val_accuracy: 0.3636363744735718

Best val_accuracy So Far: 0.6688311696052551
Total elapsed time: 00h 00m 07s


In [132]:
# Getting best optimizer values
tuner.get_best_hyperparameters()[0].values

{'num_layers': 6,
 'units0': 40,
 'activation0': 'relu',
 'dropout0': 0.4,
 'optimizer': 'adadelta',
 'units1': 16,
 'activation1': 'tanh',
 'dropout1': 0.6,
 'units2': 56,
 'activation2': 'relu',
 'dropout2': 0.2,
 'units3': 48,
 'activation3': 'relu',
 'dropout3': 0.7,
 'units4': 96,
 'activation4': 'tanh',
 'dropout4': 0.9,
 'units5': 80,
 'activation5': 'relu',
 'dropout5': 0.1,
 'units6': 8,
 'activation6': 'sigmoid',
 'dropout6': 0.8,
 'units7': 24,
 'activation7': 'tanh',
 'dropout7': 0.2}

In [133]:
model = tuner.get_best_models(num_models=1)[0]

In [134]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 40)                360       
                                                                 
 dropout (Dropout)           (None, 40)                0         
                                                                 
 dense_1 (Dense)             (None, 16)                656       
                                                                 
 dropout_1 (Dropout)         (None, 16)                0         
                                                                 
 dense_2 (Dense)             (None, 56)                952       
                                                                 
 dropout_2 (Dropout)         (None, 56)                0         
                                                                 
 dense_3 (Dense)             (None, 48)                2

In [135]:
model.fit(x_train, y_train, epochs=200, initial_epoch = 6, validation_data=(x_test,y_test))

Epoch 7/200
20/20 [==============================] - 1s 13ms/step - loss: 0.7348 - accuracy: 0.5782 - val_loss: 0.6900 - val_accuracy: 0.6688
Epoch 8/200
20/20 [==============================] - 0s 5ms/step - loss: 0.7349 - accuracy: 0.5668 - val_loss: 0.6899 - val_accuracy: 0.6688
Epoch 9/200
20/20 [==============================] - 0s 5ms/step - loss: 0.7585 - accuracy: 0.5440 - val_loss: 0.6899 - val_accuracy: 0.6688
Epoch 10/200
20/20 [==============================] - 0s 4ms/step - loss: 0.7019 - accuracy: 0.6042 - val_loss: 0.6899 - val_accuracy: 0.6688
Epoch 11/200
20/20 [==============================] - 0s 5ms/step - loss: 0.7410 - accuracy: 0.5456 - val_loss: 0.6898 - val_accuracy: 0.6753
Epoch 12/200
20/20 [==============================] - 0s 4ms/step - loss: 0.7501 - accuracy: 0.5342 - val_loss: 0.6898 - val_accuracy: 0.6818
Epoch 13/200
20/20 [==============================] - 0s 4ms/step - loss: 0.7641 - accuracy: 0.4984 - val_loss: 0.6898 - val_accuracy: 0.6818
Epoch 14